In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cptac
en = cptac.Ucec()

clinical_data = en.get_clinical('mssm')
print(clinical_data.columns.to_list()) # a ton of data columns
clinical_data['tumor_stage_pathological'].unique()
clinical_data['tumor_stage_pathological'].value_counts()
print(f"The minimum tumor size is {clinical_data['tumor_size_cm'].min()}")
print(f"The maximum tumor size is {clinical_data['tumor_size_cm'].max()}")
print(f"The average tumor size is {clinical_data['tumor_size_cm'].mean()}")
#en.get_acetylproteomics('umich')
#Set desired attribute to variable 'clinical_attribute'
clinical_attribute = "tumor_size_cm"

#Join attribute with proteomics dataframe
clinical_and_proteome = en.join_metadata_to_omics(metadata_name='clinical',
                                                    omics_name='proteomics',
                                                    omics_source='umich',
                                                    metadata_source='mssm',
                                                    metadata_cols=clinical_attribute)




['tumor_code', 'discovery_study', 'type_of_analyzed_samples', 'confirmatory_study', 'type_of_analyzed_samples', 'age', 'sex', 'race', 'ethnicity', 'ethnicity_race_ancestry_identified', 'Inferred ancestry', 'collection_in_us', 'participant_country', 'maternal_grandmother_country', 'maternal_grandfather_country', 'paternal_grandmother_country', 'paternal_grandfather_country', 'deaf_or_difficulty_hearing', 'blind_or_difficulty_seeing', 'difficulty_concentrating_remembering_or_making_decisions', 'difficulty_walking_or_climbing_stairs', 'difficulty_dressing_or_bathing', 'difficulty_doing_errands', 'consent_form_signed', 'case_stopped', 'tumor_site', 'tumor_site_other', 'tumor_laterality', 'tumor_focality', 'tumor_size_cm', 'histologic_type', 'histologic_grade', 'tumor_necrosis', 'margin_status', 'ajcc_tnm_cancer_staging_edition_used', 'pathologic_staging_primary_tumor_pt', 'pathologic_staging_regional_lymph_nodes_pn', 'number_of_lymph_nodes_examined', 'number_of_lymph_nodes_positive_for_tum

In [2]:
# Step 1: Exclude rows without tumor size
clinical_and_proteome = clinical_and_proteome.dropna(subset=["tumor_size_cm"])

# Step 2: Calculate the threshold for filtering columns with more than 30% NaN values
threshold = len(clinical_and_proteome) * 0.7

# Filter columns based on the threshold of non-NaN values
valid_columns = clinical_and_proteome.columns[
    clinical_and_proteome.isna().sum() < threshold
]

# Filter the original DataFrame to keep only the valid columns
filtered_df = clinical_and_proteome[valid_columns]

# Calculate the correlation between "tumor_size_cm" and each analyte
correlations = filtered_df.corr(method='pearson')

# Extract the correlation values for "tumor_size_cm"
tumor_size_correlations = correlations["tumor_size_cm"].drop(["tumor_size_cm"])

# Find the top 5 analytes with the highest correlation
top_corr_analytes = tumor_size_correlations.nlargest(5)

print("Top 5 analytes with the highest correlation to tumor size:")
print(top_corr_analytes)

Top 5 analytes with the highest correlation to tumor size:
Name
ACOT9_umich_proteomics      0.950281
HSPB7_umich_proteomics      0.850038
HSPB7_umich_proteomics      0.850038
TMPRSS4_umich_proteomics    0.789560
NBEA_umich_proteomics       0.788091
Name: tumor_size_cm, dtype: float64


In [5]:
# Get the top 10 analytes with the highest correlation
top_analytes = top_corr_analytes.index

# Create a list of columns to extract, including "tumor_size_cm"
columns_to_extract = ["tumor_size_cm"] + list(top_analytes)

# Extract these columns from the original DataFrame
top_df = clinical_and_proteome.loc[:, columns_to_extract]
# Display the new DataFrame with only "tumor_size_cm" and the top 10 analytes
print(top_df.head())
# Create a mask for selecting the exact columns from MultiIndex
columns_to_extract = [col for col in clinical_and_proteome.columns if col[1] in top_analytes or col[1] == "tumor_size_cm"]

# Extract the columns using the mask
top_df = clinical_and_proteome.loc[:, columns_to_extract]

# Display the new DataFrame with only "tumor_size_cm" and the top 10 analytes
print(top_df.head())


Empty DataFrame
Columns: [tumor_size_cm, ACOT9_umich_proteomics, ACOT9_umich_proteomics, HSPB7_umich_proteomics, HSPB7_umich_proteomics, HSPB7_umich_proteomics, HSPB7_umich_proteomics, HSPB7_umich_proteomics, HSPB7_umich_proteomics, HSPB7_umich_proteomics, HSPB7_umich_proteomics, TMPRSS4_umich_proteomics, TMPRSS4_umich_proteomics, NBEA_umich_proteomics, NBEA_umich_proteomics]
Index: []
Empty DataFrame
Columns: []
Index: [C3L-00006, C3L-00008, C3L-00032, C3L-00084, C3L-00090]


In [6]:
top_df_cleaned = top_df.dropna()

# Display the cleaned DataFrame
print(top_df_cleaned.head())


Empty DataFrame
Columns: []
Index: [C3L-00006, C3L-00008, C3L-00032, C3L-00084, C3L-00090]


In [13]:
import cptac
en = cptac.Lscc()
en.list_data_sources()
prot = en.get_proteomics("umich")
clinical = en.get_clinical('mssm')
clinical_attribute = "tumor_size_cm"
clin_and_prot = en.join_metadata_to_omics(metadata_name='clinical',
                                                    omics_name='proteomics',
                                                    omics_source='umich',
                                                    metadata_source='mssm',#)
                                                    metadata_cols=clinical_attribute)
clin_and_prot


Name,tumor_size_cm,ARF5_umich_proteomics,M6PR_umich_proteomics,ESRRA_umich_proteomics,FKBP4_umich_proteomics,NDUFAF7_umich_proteomics,FUCA2_umich_proteomics,DBNDD1_umich_proteomics,HS3ST1_umich_proteomics,SEMA3F_umich_proteomics,...,SCRIB_umich_proteomics,WIZ_umich_proteomics,PRX_umich_proteomics,LDB1_umich_proteomics,WIZ_umich_proteomics,TSGA10_umich_proteomics,RFX7_umich_proteomics,MSANTD2_umich_proteomics,SVIL_umich_proteomics,ADGRL2_umich_proteomics
Patient_ID,,,,,,,,,,,,,,,,,,,,,
C3L-00081,5.0,0.188571,0.191483,0.008152,0.564788,0.235464,-1.067834,NaN,NaN,-0.037184,...,NaN,NaN,-1.744131,-0.477826,0.143987,-0.846648,0.566704,-0.910259,NaN,-0.083542
C3L-00415,6.5,0.279194,0.090933,0.090611,0.017926,0.443648,0.610357,NaN,NaN,0.095169,...,-0.067776,0.362171,-1.372611,0.074603,-0.135047,0.049109,0.296279,0.427957,NaN,NaN
C3L-00445,5.0,0.471727,0.163861,-0.212975,0.579089,0.331020,-0.117381,-0.012115,-0.442592,0.178850,...,NaN,NaN,-1.313231,0.189654,0.145014,-1.232546,0.359494,-0.010463,NaN,NaN
C3L-00568,4.7,0.126844,0.013168,0.065744,0.141438,0.156805,-0.210740,NaN,NaN,-0.269801,...,NaN,0.209074,-1.135292,0.157129,-0.012959,-1.100473,0.192179,-0.115794,-0.999066,NaN
C3L-00603,3.0,0.863122,0.218243,0.370573,0.426401,0.450809,-0.283630,0.050006,NaN,-0.197599,...,NaN,-0.118910,-1.292992,1.391250,0.320952,-1.146583,0.421160,0.439811,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C3N-03886.N,NaN,-0.266226,-0.461862,-0.124028,-0.799347,-0.220203,0.074583,NaN,NaN,-0.561419,...,NaN,-0.352598,0.773663,0.122366,-0.191186,0.514100,NaN,0.981718,NaN,NaN
C3N-04124.N,NaN,-0.157557,-0.131754,0.171796,-0.808126,-0.251913,0.098788,NaN,-0.324405,0.115747,...,NaN,-0.495171,1.104057,-0.084635,-0.360851,0.887407,-0.455884,NaN,1.026611,NaN
C3N-04127.N,NaN,0.146387,-0.205130,-0.218190,-0.736550,-0.506242,0.087855,NaN,-0.013404,-0.208149,...,NaN,0.058459,1.009561,-0.146886,-0.198534,0.586517,-0.210785,NaN,2.321351,NaN


In [14]:
for c in list(clinical.columns):
    print(c)

tumor_code
discovery_study
type_of_analyzed_samples
confirmatory_study
type_of_analyzed_samples
age
sex
race
ethnicity
ethnicity_race_ancestry_identified
Inferred ancestry
collection_in_us
participant_country
maternal_grandmother_country
maternal_grandfather_country
paternal_grandmother_country
paternal_grandfather_country
deaf_or_difficulty_hearing
blind_or_difficulty_seeing
difficulty_concentrating_remembering_or_making_decisions
difficulty_walking_or_climbing_stairs
difficulty_dressing_or_bathing
difficulty_doing_errands
consent_form_signed
case_stopped
tumor_site
tumor_site_other
tumor_laterality
tumor_focality
tumor_size_cm
histologic_type
histologic_grade
tumor_necrosis
margin_status
ajcc_tnm_cancer_staging_edition_used
pathologic_staging_primary_tumor_pt
pathologic_staging_regional_lymph_nodes_pn
number_of_lymph_nodes_examined
number_of_lymph_nodes_positive_for_tumor_by_he_staining
clinical_staging_distant_metastasis_cm
pathologic_staging_distant_metastasis_pm
specify_distant_me

In [18]:
# Step 1: Exclude rows without tumor size
clin_and_prot = clin_and_prot.dropna(subset=["tumor_size_cm"])
print(len(clin_and_prot))

# Step 2: Calculate the threshold for filtering columns with more than 30% NaN values
threshold = len(clin_and_prot) * 0.7

# Filter columns based on the threshold of non-NaN values
valid_columns = clin_and_prot.columns[
    clin_and_prot.isna().sum() < threshold
]

# Filter the original DataFrame to keep only the valid columns
filtered_df = clin_and_prot[valid_columns]

# Calculate the correlation between "tumor_size_cm" and each analyte
correlations = filtered_df.corr(method='pearson')

# Extract the correlation values for "tumor_size_cm"
tumor_size_correlations = correlations["tumor_size_cm"].drop(["tumor_size_cm"])

# Find the top 5 analytes with the highest correlation
top_corr_analytes = tumor_size_correlations.nlargest(5)

print("Top 5 analytes with the highest correlation to tumor size:")
print(top_corr_analytes)

107
Top 5 analytes with the highest correlation to tumor size:
Name
MICAL3_umich_proteomics     0.963779
WLS_umich_proteomics        0.917994
EPS15L1_umich_proteomics    0.905514
RGS3_umich_proteomics       0.891369
MELTF_umich_proteomics      0.886070
Name: tumor_size_cm, dtype: float64


In [21]:
# Get the top 10 analytes with the highest correlation
top_analytes = top_corr_analytes.index

# Create a list of columns to extract, including "tumor_size_cm"
columns_to_extract = ["tumor_size_cm"] + list(top_analytes)
unique_columns_to_extract = pd.Index(columns_to_extract).drop_duplicates(keep='first')

# Extract these columns from the original DataFrame
top_df = clin_and_prot.loc[:, unique_columns_to_extract]
# Display the new DataFrame with only "tumor_size_cm" and the top 10 analytes
top_df = top_df.dropna(axis=1, how='all')
print(top_df)


Name        tumor_size_cm  MICAL3_umich_proteomics  MICAL3_umich_proteomics  \
Patient_ID                                                                    
C3L-00081             5.0                 0.146975                      NaN   
C3L-00415             6.5                -0.412906                      NaN   
C3L-00445             5.0                 0.333502                      NaN   
C3L-00568             4.7                 0.019696                      NaN   
C3L-00603             3.0                 0.063782                      NaN   
...                   ...                      ...                      ...   
C3N-03886             4.2                 0.730090                      NaN   
C3N-04124             3.5                 0.017623                      NaN   
C3N-04127             5.0                 0.022904                      NaN   
C3N-04155             2.5                -0.018915                      NaN   
C3N-04162             6.0                 0.052497  

In [22]:
top_df.columns

Index(['tumor_size_cm', 'MICAL3_umich_proteomics', 'MICAL3_umich_proteomics',
       'WLS_umich_proteomics', 'WLS_umich_proteomics',
       'EPS15L1_umich_proteomics', 'EPS15L1_umich_proteomics',
       'RGS3_umich_proteomics', 'RGS3_umich_proteomics',
       'MELTF_umich_proteomics', 'MELTF_umich_proteomics'],
      dtype='object', name='Name')